Report the function definitions from aruco-recognition.ipynb for including in other scripts

In [2]:
# Open the image given a path and find aruco markers
def findAruco(imgPath, ARUCO_DICT, ARUCO_PARAMETERS):
    # Read the image with the markers
    img = cv2.imread(imgPath)

    # grayscale image
    #img_bw = cv2.cvtColor(queryImg, cv2.COLOR_BGR2GRAY)
    
    #======== Detect Aruco markers ========
    corners, ids, _ = aruco.detectMarkers(img, ARUCO_DICT, parameters=ARUCO_PARAMETERS)
    
    return img, ids, corners 

In [3]:
# Return: array with selected contours, None if no markers are provided
def findRectangles(imgRect, idsR, cornersR):
    debug = 0
    # Grayscale image is requested for contour recognition
    imgRectGray = cv2.cvtColor(imgRect, cv2.COLOR_BGR2GRAY)

    # Check if at least one marker has been found
    if idsR is None or len(idsR) == 0:
        # If no marker detected, exit
        print("No marker detected!")
        return None

    # Print found arucos
    if debug:
        for i, corner in zip(idsR, cornersR):
            print('Detected aruco with ID: {}.'.format(i[0]))

    #======== Find contours in image ========
        
    # The "findContours" function nedd a binary image, so need to threeshold before
    ret, img_thresh = cv2.threshold(imgRectGray, 127, 255, 0)
    contours, hierarchy = cv2.findContours(img_thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Identify rectangular contours
    rect_cnts = []
    areas = []
    for cnt in contours:
        peri = cv2.arcLength(cnt, True)
        approx = cv2.approxPolyDP(cnt, 0.04 * peri, True)
        (x, y, w, h) = cv2.boundingRect(cnt)
        #ar = w / float(h)
        if len(approx) == 4: # shape filtering condition
            # Get the area of the rectangle, need to exclude rectangles with area less than the 
            # one of the smallest aruco
            area = cv2.contourArea(cnt)

            # Exclude rectangles with pixel area, due to some threesholding error perhaps
            if area >= 5.0:
                areas.append(area)
                rect_cnts.append(cnt) # Shape is rectangle, add to the valid list
    # Now in rect_cnts[] we have only rectangular contours

    #======== Discard the contours that do not contain any aruco (multiple markers can be present in the image)

    # Make a copy to preserve the original image, draw functions are destructive
    imgRectDraw = np.copy(imgRect)

    j = 0
    in_cnt = []
    for aruco_n, corner_n in zip(idsR, cornersR): # for every aruco marker in image...
        cnt_father = []
        corner_n = corner_n[0] # adjust array dimensionality
        for cnt in rect_cnts: # for every rectangular contour...
            dist = cv2.pointPolygonTest(cnt, (corner_n[0][0].astype(int), corner_n[0][1].astype(int)), True) # Check if top left corner of the aruco
            if dist > 1.: # if the aruco is inside the contour...
                cnt_father.append(cnt) # add the contour in list
                if debug:
                    cv2.drawContours(imgRectDraw, [cnt], -1, (0,255,0), 2) # for debug draw the contour found
        if len(cnt_father) != 0:
            in_cnt.append(cnt_father) # check next aruco

    return in_cnt

In [ ]:
def rotate_image(image, angle):
    image_center = tuple(np.array(image.shape[1::-1]) / 2)
    rot_mat = cv2.getRotationMatrix2D(image_center, angle, 1.0)
    result = cv2.warpAffine(image, rot_mat, image.shape[1::-1], flags=cv2.INTER_LINEAR)
    return result

def rotate_bound(image, angle):
    # grab the dimensions of the image and then determine the
    # center
    (h, w) = image.shape[:2]
    (cX, cY) = (w // 2, h // 2)
    # grab the rotation matrix (applying the negative of the
    # angle to rotate clockwise), then grab the sine and cosine
    # (i.e., the rotation components of the matrix)
    M = cv2.getRotationMatrix2D((cX, cY), -angle, 1.0)
    cos = np.abs(M[0, 0])
    sin = np.abs(M[0, 1])
    # compute the new bounding dimensions of the image
    nW = int((h * sin) + (w * cos))
    nH = int((h * cos) + (w * sin))
    # adjust the rotation matrix to take into account translation
    M[0, 2] += (nW / 2) - cX
    M[1, 2] += (nH / 2) - cY
    # perform the actual rotation and return the image
    return cv2.warpAffine(image, M, (nW, nH))